In [7]:
import pandas as pd
import geopandas as gpd
import os

# === BASE DIRECTORY ===
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# === FILE PATHS ===
roads_path = os.path.join(base_dir, "street_network.shp")

# === LOAD SHAPEFILE ===
roads = gpd.read_file(roads_path)

# === CONVERT RELEVANT COLUMNS TO NUMERIC ===
numeric_cols = ['SPEED', 'qDirLanes', 'qNoAccess', 'qExclude', 'BL_WIDTH', 'BL_REACH', 'PARKALONG', 'ILLPARKING', 'StOperNEU', 'ADT_Infer', 'FEDERALFUN']
roads[numeric_cols] = roads[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [8]:
# === FUNCTION TO CALCULATE LTS_2025b ===
def calculate_LTS_2025b(row):
    """Calculate LTS_2025b based on road characteristics."""
    
    protected = row['qProtected']
    no_access = row['qNoAccess']
    exclude = row['qExclude']
    speed = row['SPEED']
    dir_lanes = row['qDirLanes']
    bl_width = row['BL_WIDTH']
    bl_reach = row['BL_REACH']
    parkalong = row['PARKALONG']
    bike_type = row['bike_type2']
    st_oper_neu = row['StOperNEU']
    adt = row['ADT_Infer']
    fed_fun = row['FEDERALFUN']

    
    if no_access in (1, 98, 99):
        return 6  # freeway (ramps)
    elif exclude in (1, 5) or (bike_type and bike_type in ("WALK_YR_BIKE")):
        return 5  # cemetary, private property, peds only, etc.
    elif fed_fun == 0 and exclude == 0:
        return 1  # Parks, alleys, bike paths, streets with almost no traffic
    elif bike_type in ("SUP", "SUP_NAT", "SUP_MINOR", "SBL", "MIX_SCONTRA", "SBL_LEFT", "CARFREE"):
        return 1  # Separated from traffic
    elif protected == 1:
        return 1 # Separated from traffic
    
    elif ((bl_width >= 4) or bike_type in ("BL", "BL_LEFT", "SBL_BL", "BL_MIX", "MIX_CONTRA", "BL_BUF", "BL_BUF_LEFT", "BUS_BL", "BUS_BL_LEFT", "BL_PK_BUS_BL")) and (parkalong == 0):
        # Conventional bike lanes NOT adjacent to parking
        if (
            (dir_lanes >= 3 and speed > 38.5) or 
            (dir_lanes == 2 and speed > 43.5 and bl_width < 6) or
            (dir_lanes <= 1 and speed > 48.5 and bl_width < 6)
        ):
            return 4
        elif (dir_lanes >= 3) or (speed > 38.5):
            return 3
        elif (
            dir_lanes == 2 or
            speed > 33.5 or
            (dir_lanes == 1 and bl_width < 6) or
            (dir_lanes == 1 and speed > 33.5)
        ):
            return 2
        elif (dir_lanes <= 1) and (bl_width >= 6):
            return 1
        else:
            return 98 # For other cases that do not meet conditions
        
    elif (bl_width >= 4 or bike_type in ("BL", "SBL_BL", "BL_LEFT", "BL_BUF", "MIX_CONTRA", "BL_PK_BUS_BL")) and (parkalong == 1) and (bl_reach >= 12):
        # Conventional bike lanes adjacent to parking
        if (dir_lanes <= 1 and speed <= 28.5 and bl_reach >= 15):
            return 1
        elif (
            (dir_lanes <= 1 and speed <= 38.5 and bl_reach >= 15) or
            (dir_lanes <= 1 and speed <= 33.5 and bl_reach < 15) or
            (dir_lanes > 1 and speed <= 28.5 and st_oper_neu == 1 and bl_reach >= 15) or
            (dir_lanes == 2 and speed <= 28.5 and bl_reach >= 15)
        ):
            return 2
        else:
            return 3

    elif (
        (bl_width < 4) or 
        ((bl_width >= 4) and (bl_reach == 1)) or 
        ((bl_width >= 4) and (parkalong > 0) and (bl_reach < 12)) or
        bike_type in ("SBL_MIX", "BL_MIX", "SLM", "SLMTC")
    ):
        # Mixed traffic conditions
        if (
            (dir_lanes >= 3 and speed > 28.5) or 
            (dir_lanes == 2 and adt > 8_000 and speed > 28.5) or 
            (dir_lanes == 2 and adt <= 8_000 and speed > 38.5)
        ):
            return 4
        elif (
            (dir_lanes >= 2) or 
            (speed > 38.5) or 
            (dir_lanes == 1 and adt > 3_000) or 
            (dir_lanes == 1 and adt > 1_500 and speed > 23.5)
        ):
            return 3
        elif (
            (dir_lanes == 1 and adt > 1_000) or 
            (dir_lanes == 1 and speed > 28.5) or 
            (dir_lanes == 0 and adt > 1_500) or 
            (dir_lanes == 0 and speed > 28.5)
        ):
            return 2
        elif (
            (dir_lanes == 1 and speed <= 28.5) or 
            (dir_lanes == 0 and speed <= 28.5)
        ):
            return 1
        else:
            return 99  # For other cases that do not meet conditions

    else:
        return 96  # Default return for other cases

# === CALCULATE LTS_2025b FOR ROADS ===
roads['LTS_2025b'] = None  # Initialize the column
roads['LTS_2025b'] = roads.apply(calculate_LTS_2025b, axis=1)

# === SAVE THE UPDATED ROADS SHAPEFILE ===
roads.to_file(roads_path, driver="ESRI Shapefile")
 
# Output summary
print("LTS_2025b calculation complete.")
print(roads['LTS_2025b'].value_counts(dropna=False))

LTS_2025b calculation complete.
LTS_2025b
1     18473
3      6289
5      6283
2      2632
6      1885
4      1148
98       86
Name: count, dtype: int64


In [9]:
roads['bike_type2'].value_counts(dropna=False)

bike_type2
None            30652
BL               1434
SUP              1276
SLM              1125
SBL               812
SUP_MINOR         530
BL_MIX            216
BL_BUF            211
BUS_BL            103
WALK_YR_BIKE       79
SUP_NAT            75
SLMTC              54
SBL_BL             52
MIX_CONTRA         38
SBL_MIX            33
BL_PK_BUS_BL       29
CARFREE            20
BL_LEFT            18
MIX_SCONTRA        11
SBL_LEFT           11
BUS_BL_LEFT         8
BL_BUF_LEFT         4
                   3
CFBS                2
Name: count, dtype: int64